# Exercício Aula 03 - Distância e Correlações

Author: Gabriel Van Loon
Prof.:  Francisco Aparecido Rodrigues
Universidade de São Paulo, São Carlos, Brasil.


## Definindo as Funções e Bibliotecas utilizadas

In [1]:
import collections
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx

from scipy.stats import pearsonr

Funções vistas na aula do exercício 03

In [26]:
def short_path_avg_var(G):
    dists = np.zeros(len(G)*(len(G)-1))
    paths = np.zeros((len(G),len(G)))    
    aux = 0
    for i in range(len(G)):
        print("Calculating row:", i)
        for j in range(i+1, len(G)):
            if i != j:
                dists[aux] = dists[(aux+1)] = len(nx.shortest_path(G,i,j))-1
                paths[i,j] = paths[j,i] = dists[aux]
                aux        += 2
                
    return np.average(dists), np.var(dists), paths

def pearson_degree_correlation(G):
    ki = []
    kj = []
    for i in range(0,len(G.nodes())):
        for j in range(0, len(G.nodes())):
            if(G.has_edge(i,j) == True):
                ki.append(G.degree(i))
                kj.append(G.degree(j))
    
    corr, _ = pearsonr(ki, kj)
    return corr, ki, kj

Funções vistas na aula do exercício 02

In [3]:
def network_degree(G):
    vk = np.array([v for v in dict(G.degree()).values()])
    return vk, avg_degree

# Optimized version by networkx documentation
def degree_distribution(G, normalize=False):
    degree_sequence = sorted([d for n, d in G.degree()], reverse=True)
    degree_count    = collections.Counter(degree_sequence)
    
    if normalize:
        total_sum = sum(degree_count.values())
        for degree in degree_count:
            degree_count[degree] = degree_count[degree]/total_sum
            
    return degree_count

# Francisco originals
def degree_distribution_original(G):
    vk = dict(G.degree())
    vk = list(vk.values())  # we get only the degree values
    vk = np.array(vk)
    maxk = np.max(vk)
    mink = np.min(vk)
    kvalues= np.arange(0,maxk+1) # possible values of k
    Pk = np.zeros(maxk+1) # P(k)
    for k in vk:
        Pk[k] = Pk[k] + 1
    Pk = Pk/sum(Pk) # the sum of the elements of P(k) must to be equal to one
    return kvalues,Pk

def momment_of_degree_distribution(G,m):
    M = 0
    N = len(G)
    for i in G.nodes:
        M = M + G.degree(i)**m
    M = M/N
    return M

def network_variance(G):
    variance = momment_of_degree_distribution(G,2) - momment_of_degree_distribution(G,1)**2
    return variance

def shannon_entropy(G):
    k,Pk = degree_distribution_original(G)
    H = 0
    for p in Pk:
        if(p > 0):
            H = H - p*np.math.log(p, 2)
    return H

def normalized_shannon_entropy(G):
    k,Pk = degree_distribution_original(G)
    H = 0
    for p in Pk:
        if(p > 0):
            H = H - p*np.math.log(p, 2)
    return H/np.math.log(len(G),2)

def network_complexity_coef(G):
    return (momment_of_degree_distribution(G,2) / momment_of_degree_distribution(G,1))

# Just to remember the calls
# nx.transitivity(G)
# nx.average_clustering(G)

## 1) Para a rede “Hamsterster”, calcule a média dos menores caminhos e o diâmetro. Use apenas o maior componente da rede e remova ciclos ou auto-conexões.

In [4]:
# Read and prepare the network
G = nx.read_edgelist('data/hamsterster.txt', nodetype=int)
# Grafo não direcionado
G = G.to_undirected()
# Remover auto-loops
G.remove_edges_from(nx.selfloop_edges(G))
# Escolhe maior componente
Gcc = sorted(nx.connected_components(G), key=len, reverse=True)
G = G.subgraph(Gcc[0])
# Renomeia os vértices
G = nx.convert_node_labels_to_integers(G, first_label=0)

In [5]:
print("Average shortest path length:", "%3.4f"%nx.average_shortest_path_length(G))
print('Network diameter:', nx.diameter(G))

Average shortest path length: 3.4526
Network diameter: 14


## 2) Considere a rede “USairport500” e calculea média e variância dos menores caminhos. Use apenas o maior componente da rede e remova ciclos ou auto-conexões.

In [37]:
# Read and prepare the network
G = nx.read_edgelist('data/USairport500.txt', nodetype=int)
# Grafo não direcionado
G = G.to_undirected()
# Remover auto-loops
G.remove_edges_from(nx.selfloop_edges(G))
# Escolhe maior componente
Gcc = sorted(nx.connected_components(G), key=len, reverse=True)
G = G.subgraph(Gcc[0])
# Renomeia os vértices
G = nx.convert_node_labels_to_integers(G, first_label=0)

In [ ]:
avg, var, short_path_dist = short_path_avg_var(G)

In [20]:
print("Avg shortest path:", "%3.4f"%avg)
print("Variance shortest path:", "%3.4f"%var)

Avg shortest path: 2.9910
Variance shortest path: 0.8175


## 3) Para a rede “USairport500”, calcule a entropia de Shannon da distribuição dos menores caminhos. Use logaritmo na base 2 e considere apenas o maior componente da rede

In [53]:
# Criando a distribuição de distancias
path_dist = np.zeros(nx.diameter(G)+1)

for i in range(0,len(G.nodes())):
    for j in range(0, len(G.nodes())):
        dij = int(short_path_dist[i,j])
        path_dist[dij] += 1.0
        
path_dist = path_dist/np.sum(path_dist)

from scipy.stats import entropy
print('Shannon Entropy:', entropy(path_dist, base=2))

Shannon Entropy: 1.9007137451744505


In [56]:
path_dist

array([2.00000e-03, 2.38400e-02, 2.80136e-01, 4.31960e-01, 2.09184e-01,
       4.90560e-02, 3.72800e-03, 9.60000e-05])

## 4) Calcule o coeficiente de assortatividade da rede Advogato. Considere apenas o maior componente.

In [21]:
# Read and prepare the network
G = nx.read_edgelist('data/advogato.txt', nodetype=int, data=(('weight',float),) )
# Grafo não direcionado
G = G.to_undirected()
# Remover auto-loops
G.remove_edges_from(nx.selfloop_edges(G))
# Escolhe maior componente
Gcc = sorted(nx.connected_components(G), key=len, reverse=True)
G = G.subgraph(Gcc[0])
# Renomeia os vértices
G = nx.convert_node_labels_to_integers(G, first_label=0)

In [22]:
print("Assortativity = ","%3.4f"%nx.degree_assortativity_coefficient(G))

Assortativity =  -0.0957


## 5) Calcule o coeficiente de correlação de Pearson entre o grau médio dos vizinhos e o grau de cada vértice para a rede “word_adjacencies”. Isto é, entre k e knn(k). Use apenas o maior componente.Considere o exemplo da aula. 

In [57]:
# Read and prepare the network
G = nx.read_edgelist('data/word_adjacencies.txt', nodetype=int, data=(('weight',float),) )
# Grafo não direcionado
G = G.to_undirected()
# Remover auto-loops
G.remove_edges_from(nx.selfloop_edges(G))
# Escolhe maior componente
Gcc = sorted(nx.connected_components(G), key=len, reverse=True)
G = G.subgraph(Gcc[0])
# Renomeia os vértices
G = nx.convert_node_labels_to_integers(G, first_label=0)

In [61]:
corr, ki, kj = pearson_degree_correlation(G)
print('Pearsons correlation:', corr)

Pearsons correlation: -0.12934785343900132


In [63]:
knn = []
for i in G.nodes():
    aux =  nx.average_neighbor_degree(G, nodes = [i])
    knn.append(float(aux[i]))
knn = np.array(knn)
print("Average degree of the neighborhood of the network:", "%3.2f"%np.mean(knn))

Average degree of the neighborhood of the network: 14.76


In [66]:
vk = dict(G.degree())
vk = list(vk.values())

knnk = list()
ks = list()
for k in np.arange(np.min(vk), np.max(vk)+1):
    aux = vk == k
    if(len(knn[aux]) > 0):
        av_knn = np.mean(knn[aux]) #average clustering among all the nodes with degree k
        knnk.append(av_knn)
        ks.append(k)
        


rho = np.corrcoef(ks, knnk)[0,1]
print('Pearson correlation coefficient:', rho)

Pearson correlation coefficient: -0.710832214935246
